In [ ]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from IPython.display import display


In [ ]:
%load_ext autoreload
%autoreload 2


### Load official statistics:

**2018 ACS 1-Year Estimates Subject Tables :**

- [`S0101`](https://api.census.gov/data/2018/acs/acs1/subject/groups/S0101.html):
"Total population : SELECTED AGE CATEGORIES".

- [`S1901`](https://api.census.gov/data/2018/acs/acs1/subject/groups/S1901.html):
"Households : Total".

- [`S2301`](https://api.census.gov/data/2018/acs/acs1/subject/groups/S2301.html): 
"Labor Force Participation Rate".

- [`B28010`](https://api.census.gov/data/2018/acs/acs1/subject/groups/S2801.html): 
"Has one or more types of computing devices : Smartphone".

**ACS Cartographic Boundary Shapefile:**

- [`cb_2018_us_county_500k.gdb`](https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.2018.html):
"Land area by county".

**2011-2015 5-Year ACS Commuting Flows (from Micro-data):***

- [`Table 1`](https://www.census.gov/data/tables/2015/demo/metro-micro/commuting-flows-2015.html): 
"Residence County to Workplace County Commuting Flows for the United States and Puerto Rico Sorted by Residence Geography: 5-Year ACS, 2011-2015".

**National Center for Health Statistics (NCHS_ Urban-Rural Classification:**

- [`NCHSurbruralcodes`](https://www.cdc.gov/nchs/data_access/urban_rural.htm):
"NCHS Urban-Rural Classification Scheme for Counties : 2013 codes".

**Federal Highway Administration:**

[`National Household Travel Survey`](https://nhts.ornl.gov/person-miles):
"Person Miles of Travel"

**Pew Reserach:**

[`Mobile Phone Factsheet`](https://www.pewresearch.org/internet/fact-sheet/mobile/):


In [ ]:
CENSUS_DATA_ROOT = './data/clean_census_data/'
PEW_RESEARCH_DATA_ROOT = './data/pew_research_data/'


In [ ]:
# Import population pyramid:
census_population = pd.read_csv(CENSUS_DATA_ROOT+"ACSST1Y2018_S0101_population.csv")

# Import employment data:
census_employment = pd.read_csv(CENSUS_DATA_ROOT+"ACSST1Y2018_S2301_employment.csv")

# Import income data:
census_income = pd.read_csv(CENSUS_DATA_ROOT+"ACSST1Y2018_S2503_income.csv")

# Import phone onwership data:
census_internet = pd.read_csv(CENSUS_DATA_ROOT+"ACSST1Y2018_S2801_internet.csv")

# Import geography data:
census_geography = pd.read_csv(CENSUS_DATA_ROOT+"cb_2018_us_county_500k.csv")

# Import commuting data:
census_commuting = pd.read_csv(CENSUS_DATA_ROOT+"ACSCommutingFlows.csv")

# Import urban/rural data:
census_urban_rural = pd.read_csv(CENSUS_DATA_ROOT+"NCHSURCodes2013_urbanrural.csv")

# Import urban/rural data:
pew_mobile = pd.read_csv(PEW_RESEARCH_DATA_ROOT+"MobilePhoneFactsheet2019.csv")


In [ ]:
# Designate column groups:
income_lowest_cols = [
    'Less than $10,000',
    '$10,000 to $14,999', 
    '$15,000 to $24,999',
]
income_midlow_cols = [
    '$25,000 to $34,999',
    '$35,000 to $49,999',
]
income_midhigh_cols = [
    '$50,000 to $74,999',
]
income_highest_cols = [
    '$75,000 to $99,999',
    '$100,000 to $149,999',
    '$150,000 to $199,999',
    '$200,000 or more',
]
income_low_cols = income_lowest_cols+income_midlow_cols
income_high_cols = income_midhigh_cols+income_highest_cols


In [ ]:
# Get geography data:
data = census_geography[['AFFGEOID','NAME','ALAND']].rename(columns={
    'AFFGEOID' : 'id',
    'NAME' : 'county_name',
    'ALAND' : 'area_square_meters',
})
# Get population data:
data = data.merge(
    census_population[['id','Total Population']],
    left_on=['id'], right_on=['id'],
)
# Get employment data:
data = data.merge(
    census_employment,
    left_on=['id'], right_on=['id'],
)
# Get urban/rural data:
data = data.merge(
    census_urban_rural.rename(columns={
        '2013 code' : 'urbanrural_code_2013', 'type_2013' : 'urbanrural_type_2013',
    })[[
        'id','urbanrural_code_2013','urbanrural_type_2013'
    ]],
    left_on=['id'], right_on=['id'],
)
# Add commuting data:
data = data.merge(
    census_commuting[['id','State Name','County Name','number_work_in_county','number_work_out_of_county']],
    left_on=['id'], right_on=['id'],
)
# Add income data:
data = data.merge(
    census_income[[
        'id',
        'Median income (dollars)',
        'Mean income (dollars)',
    ]+income_high_cols+income_low_cols],
    left_on=['id'], right_on=['id'],
)
# Add phone ownership:
data = data.merge(
    census_internet[['id','smartphone_ownership']],
    left_on=['id'], right_on=['id'],
)
# Compute metrics:
data['pct_above_income_50000'] = census_income[income_high_cols].sum(axis=1)/100
data['pop_density'] = data['Total Population']/(data['area_square_meters']/1e6)
data['pct_work_out_of_county'] = data['number_work_out_of_county']/(
    data['number_work_in_county']+data['number_work_out_of_county']
)/100
data['Labor Force Participation Rate'] = data['Labor Force Participation Rate'].replace('N',np.nan)
data['Labor Force Participation Rate'] = data['Labor Force Participation Rate'].astype(float)/100
data['urban_rural'] = np.where(data['urbanrural_code_2013'].isin([1,2,3]),'urban','rural')
# Rename columns:
data = data.rename(columns={'id':'county_id'})

data


### Build `location_attributes` table:


- For `population`, we used total population in age/gender breakdown (from ACS). 

- For `density`, we classifed counties as `urban` if they had code 1, 2, or 3 in the NCHS data and `rural` if they had code 4, 5, or 6 (NCHS classification).

- For `wealth_rate`, we took the proportion of population in household with income of 50,000 or more (from ACS).

- For `employment_rate`, we used labor force participation rate (from ACS).


In [ ]:
new_column_names = {
    'county_id' : 'location_id',
    #'county_name' : 'location_county',
    'County Name' : 'location_county',
    'State Name' : 'location_state',
    'Geographic Area Name' : 'location_name',
    'Total Population' : 'population',
    'urban_rural' : 'density',
    'Labor Force Participation Rate' : 'employment_rate',
    'pct_above_income_50000' : 'wealth_rate',
}
data_columns = [
    'location_id',
    'location_county',
    'location_state',
    'location_name',
    'population',
    'density',
    'employment_rate',
    'wealth_rate',
]
location_attributes = data.copy().rename(columns=new_column_names)[data_columns]
location_attributes


### Build `location_profiles` table:

- We calculated population density by using the county boundary shapefiles and used it to calibrate parameters of work/social/grocery travel distances.


In [ ]:
def estimate_smartphone_breakdown(row):
    
    """
        Estimates this location's breakdown of smartphone ownership by income bracket
        using income breakdown in overall population (from Pew Research),
        and this location's income brackets and overall ownernship (from ACS).
    """

    # Get overall phone ownership by income level:
    pew_income_groups = ['Less than $30,000','$30,000-$49,999','$50,000-$74,999','$75,000+']
    overall_ownership = []
    for pew_income_group in pew_income_groups:
        rate = pew_mobile.set_index(['Category']).loc[pew_income_group]['Smartphone']
        rate = float(rate.replace('%','').strip())/100
        overall_ownership.append(rate)
    overall_ownership = np.array(overall_ownership)

    # Get population by income level for this location:
    location_population = []
    income_col_groups = [income_lowest_cols,income_midlow_cols,income_midhigh_cols,income_highest_cols]
    for income_col_group in income_col_groups:
        count = row['Total Population']*row[income_col_group].sum()/100
        location_population.append(count)
    location_population = np.array(location_population)

    # Get overall phone ownership for this location:
    location_ownership = row['smartphone_ownership']

    # Calculate scaling factor to maintain overall ownership rate for this location:
    unscaled_estimate = (overall_ownership @ location_population) / location_population.sum()
    scaling_factor = location_ownership / unscaled_estimate

    # Estimate ownership rate in each income group for this location using (rescaled) overall breakdown:
    location_ownership_estimated = overall_ownership * scaling_factor

    # Check that overall rate from this estimate is similar to the known rate for this location:
    check = (location_ownership_estimated @ location_population) / location_population.sum()
    assert np.abs(location_ownership-check)<0.0001
    
    # Group highest and lowest income breackes to get two-level ownership rate:
    people = location_population
    owners = location_ownership_estimated * location_population
    people = np.array([people[0:2].sum(),people[2:4].sum()])
    owners = np.array([owners[0:2].sum(),owners[2:4].sum()])
    result = owners/people
    
    return result


In [ ]:
location_profiles = []
wealth_status_values = [0,1]
employment_status_values = [0,1]
for i,row in data.iterrows():
    new_row = row.copy().rename(new_column_names)
    
    smartphone_lowincome, smartphone_highincome = estimate_smartphone_breakdown(row)
    smartphone_ownership = { 0 : smartphone_lowincome, 1 : smartphone_highincome }
    
    for wealth_status in wealth_status_values:
        for employment_status in employment_status_values:
            profile = {
                'location_id' : new_row['location_id'],
                'location_county' : new_row['location_county'],
                'location_state' : new_row['location_state'],
                'location_name' : new_row['location_name'],
                'wealth_status' : wealth_status,
                'employment_status' : employment_status,
                'phoneownership_rate' : smartphone_ownership[wealth_status],
            }
            location_profiles.append(profile)
location_profiles = pd.DataFrame(location_profiles)
location_profiles
